#add no cluster
#com.databricks:spark-xml_2.12:0.16.0

In [0]:
# ================================================================
# Bronze - Importar XML Limites dos Biomas (IBGE)
# ================================================================
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.appName("Bronze_LimitesBiomas_XML").getOrCreate()

In [0]:
# ================================================================
# Parâmetro recebido via Job Databricks
# ================================================================
dbutils.widgets.text("data_ref_carga", "")
data_ref_carga = dbutils.widgets.get("data_ref_carga")

if not data_ref_carga:
    raise ValueError("❌ Parâmetro 'data_ref_carga' não informado (formato esperado: yyyy-MM-dd)")

print(f"🗓️ Data de referência da carga: {data_ref_carga}")


In [0]:
# ================================================================
# Configurações
# ================================================================
catalog = "amazonia_catalog"
schema = "b_tbra"
table_name = "e_lim_bio"
path_raw = "/Volumes/amazonia_catalog/raw/raw_tbra/Limites_dos_Biomas.xml"

In [0]:
# ================================================================
# 1️⃣ Ler o XML
# ================================================================
df_xml = (
    spark.read.format("xml")
    .option("rowTag", "gmd:MD_Metadata")
    .load(path_raw)
)

df_xml = df_xml.withColumn("data_ref_carga", F.lit(data_ref_carga))

print("✅ XML lido e coluna data_ref_carga adicionada.")

In [0]:
# ================================================================
# 2️⃣ Gravar tabela Delta particionada
# ================================================================
full_table_name = f"{catalog}.{schema}.{table_name}"

(
    df_xml.write
    .format("delta")
    .mode("append")  # mantém histórico de partições
    .partitionBy("data_ref_carga")
    .saveAsTable(full_table_name)
)

print(f"✅ Tabela Bronze gravada com partição: {full_table_name}")

In [0]:
# ================================================================
# 3️⃣ Verificação
# ================================================================
display(spark.sql(f"SELECT DISTINCT data_ref_carga FROM {full_table_name} ORDER BY data_ref_carga DESC")